Code for Question 1

In [1]:
import numpy as np
import numpy.linalg as la
A = np.array([[1,2,3,4],[2,4,-4,8],[-5,4,1,5],[5,0,-3,-7.0]])
ATA = A.transpose() @ A
w, v = la.eig(ATA)
print(f'The vector that correspond with the biggest eigan '
      f'value is \n x = {v.transpose()[0]}')

The vector that correspond with the biggest eigan value is 
 x = [-0.29618621  0.35616716  0.06730298  0.88367923]


Question 4 - least squares

In [2]:
b = np.array([6.0,1,5,2])
A = np.array([[2.0,1,2],[1,-2,1],[1,2,3],[1,1,1]])
ATA = A.transpose()@A

In [3]:
def forward_sub(L,b):
    L = np.copy(L)
    b = np.copy(b)
    for i in range(len(L)):
        b[i] = b[i] / L[i,i]
        L[i][i] = 1
        for j in range(i+1,len(L)):
            b[j] = b[j] - b[i] * (L[j][i])
            L[j] = L[j] - L[i] * (L[j][i])
    return b


In [4]:
def backward_sub(LT,y):
    LT = np.copy(LT)
    y = np.copy(y)
    for i in reversed(range(len(LT))):
        y[i] = y[i] / LT[i,i]
        LT[i][i] = 1
        for j in range(0,i):
            y[j] = y[j] - y[i] * (LT[j][i])
            LT[j] = LT[j] - LT[i] * (LT[j][i])
    return y

Section (a) - using Cholesky Factorization

In [5]:
ATB = A.transpose() @ b
L = np.linalg.cholesky(ATA)
y = forward_sub(L,ATB)
x = backward_sub(L.transpose(),y)
print(f'the vector that we found is x={x}')




the vector that we found is x=[1.7 0.6 0.7]


Section (b) using QR , SVD

In [6]:
Q,R = np.linalg.qr(A)
y = forward_sub(R.transpose(),ATB)
x_qr = backward_sub(R,y)
print(f'The vector that we found using qr factorization is x={x_qr}')






The vector that we found using qr factorization is x=[1.7 0.6 0.7]


now we will solve the problem using SVD

In [7]:
def sigma_solver(s,UTb):
    s = np.copy(s)
    UTb = np.copy(UTb)
    y = np.array([UTb[i]/s[i][i] for i in range(len(UTb)-1)])
    return y

In [8]:
u, s, vt = np.linalg.svd(A, full_matrices=True)
# follow the equation : s@vt@x = u^t @ b
s = np.diag(s)
st = np.vstack((s, [0, 0, 0]))
y = sigma_solver(s,u.transpose() @ b)
x_svd = vt.transpose() @ y
print(f'The vector that we found using svd decomposition??? is x={x_svd}')


The vector that we found using svd decomposition??? is x=[1.7 0.6 0.7]


Section (c):

In [9]:
print(f'The residual of the least squares system is: {A@x_svd - b}')
print(f'A.transpose() @ r = {A.transpose()@(A@x_svd - b)}')

The residual of the least squares system is: [-6.00000000e-01  2.00000000e-01  1.77635684e-15  1.00000000e+00]
A.transpose() @ r = [3.55271368e-15 1.77635684e-15 7.10542736e-15]


it is not suprising because we know the noraml equation : A.transpose()*A*x =A.transpose()*b
so A.transpose() @ (A @ x - b)  = 0 and (A @ x - b) =r


Section (d):

In [10]:
W = np.diag([1000, 1, 1, 1])
ATWA = A.transpose() @ W @ A
x_wighted = np.linalg.inv(ATWA) @ A.transpose() @ W @ b
r_new = A @ x_wighted - b
print(f'r1 = {r_new[0]}')
print(np.abs(r_new[0]) < 10 ** -3)


r1 = -0.0008074128194239805
True


Question 5

In [11]:
def gram_schmidt_qr(A):
    AT = A.transpose()
    R = np.zeros((len(A[0]),len(A[0])))
    Q = np.zeros((len(AT),len(AT[0])))
    #init
    R[0][0] = la.norm(AT[0])
    Q[0] = AT[0] / R[0][0]
    for i in range(1,len(AT)):
        Q[i] = AT[i]
        for j in range(0,i):
            R[j][i] = Q[j].transpose() @ AT[i] #verify that is float not vector
            Q[i] -= R[j][i] * Q[j]
        R[i][i] = la.norm(Q[i])
        Q[i] = Q[i] / R[i][i]
    return Q.transpose(), R

def gram_schmidt_qr_modified(A):
    AT = A.transpose()
    R = np.zeros((len(A[0]),len(A[0])))
    Q = np.zeros((len(AT),len(AT[0])))
    #init
    R[0][0] = la.norm(AT[0])
    Q[0] = AT[0] / R[0][0]
    for i in range(1,len(AT)):
        Q[i] = AT[i]
        for j in range(0,i):
            R[j][i] = Q[j].transpose() @ Q[i] #verify that is float not vector
            Q[i] -= R[j][i] * Q[j]
        R[i][i] = la.norm(Q[i])
        Q[i] = Q[i] / R[i][i]
    return Q.transpose(), R

In [12]:
A1 = np.array([[1,1,1],[1,0,0],[0,1,0],[0,0,1]])
A2 = np.array([[1,1,1],[1e-10,0,0],[0,1e-10,0],[0,0,1e-10]])
print(f'The QR factorization of the with epslion = 1')
Q11,R11 = gram_schmidt_qr(A1)
print(f'regular: \n Q= \n  {Q11} \n\n R=  \n {R11} \n')
Q12,R12 = gram_schmidt_qr_modified(A1)
print(f'modified:\n  Q= \n  {Q12} \n R=  \n {R12} \n\n')
print(f'The QR factorization of the with epslion = 1e-10')
Q21,R21 = gram_schmidt_qr(A2)
print(f'regular: \n Q= \n  {Q21} \n\n R=  \n {R21} \n')
Q22,R22 = gram_schmidt_qr_modified(A2)
print(f'modified: \n Q= \n  {Q22} \n R=  \n {R22} ')
I = np.diag([1,1,1])

The QR factorization of the with epslion = 1
regular: 
 Q= 
  [[ 0.70710678  0.40824829  0.28867513]
 [ 0.70710678 -0.40824829 -0.28867513]
 [ 0.          0.81649658 -0.28867513]
 [ 0.          0.          0.8660254 ]] 

 R=  
 [[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]] 

modified:
  Q= 
  [[ 0.70710678  0.40824829  0.28867513]
 [ 0.70710678 -0.40824829 -0.28867513]
 [ 0.          0.81649658 -0.28867513]
 [ 0.          0.          0.8660254 ]] 
 R=  
 [[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]] 


The QR factorization of the with epslion = 1e-10
regular: 
 Q= 
  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e-10 -7.07106781e-01 -7.07106781e-01]
 [ 0.00000000e+00  7.07106781e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  7.07106781e-01]] 

 R=  
 [[1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.41421356e-10 0.0000

In [13]:
print('\n for epslion = 1, forbinus norm for Q.transpose()@Q - I')
print(f'regular : {la.norm(Q11.transpose()@Q11 - I)}')
print(f'modified : {la.norm(Q12.transpose()@Q12 - I)}')
print('\n for epslion = 1e-10, forbinus norm for Q.transpose()@Q - I')
print(f'regular : {la.norm(Q21.transpose()@Q21 - I)}')
print(f'modified : {la.norm(Q22.transpose()@Q22 - I)}')


 for epslion = 1, forbinus norm for Q.transpose()@Q - I
regular : 5.319287782567757e-16
modified : 4.987305196443834e-16

 for epslion = 1e-10, forbinus norm for Q.transpose()@Q - I
regular : 0.7071067811865477
modified : 1.1547005383855976e-10



We can see that modified algorithm produces a better QR factorization
becuase with epsilon = 1e-10 we got forbinus norm which was closer to 0, then
the one we got using the regular algorithm.